In [2]:
pip install azure-storage-blob

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import requests
import pandas as pd
import json
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
from azure.core.pipeline.transport import HttpResponse

In [4]:
def get_data(date, time):

    url = "https://data.gharchive.org/"+ str(date) +'-'+ str(time) +'.json.gz'
    print(url)

    response = requests.get(url)

    if response.status_code == 200:

       with open("cache.json.gz", "wb") as file:

            file.write(response.content)

       print("File downloaded successfully.")

    else:

       print("Error downloading the file.")


In [5]:
def format_payload(row):
    formatted_payload = {
        'action': row['action'],
        'issue.url': row['issue']['url'],
        'issue.number': row['issue']['number'],
        'issue.title': row['issue']['title'],
        'issue.user.login': row['issue']['user']['login'],
        'issue.user.type': row['issue']['user']['type'],
        'issue.user.site_admin': row['issue']['user']['site_admin'],
        'issue.labels': json.dumps(row['issue']['labels']),
        'issue.state': row['issue']['state'],
        'issue.locked': row['issue']['locked'],
        'issue.milestone': json.dumps(row['issue']['milestone']),
        'issue.comments': row['issue']['comments'],
        'issue.created_at': row['issue']['created_at'],
        'issue.updated_at': row['issue']['updated_at'],
        'issue.closed_at': row['issue']['closed_at'],
        'issue.author_association': row['issue']['author_association'],
        'issue.active_lock_reason': row['issue']['active_lock_reason'],
        'issue.body': row['issue']['body'],
        'issue.performed_via_github_app': json.dumps(row['issue']['performed_via_github_app']),
        'issue.state_reason': row['issue']['state_reason']
    }
    return formatted_payload

In [6]:
def normalize_data():
  filename = "cache.json.gz"
  df = pd.read_json(filename, compression="gzip", lines=True).drop('id', axis=1)
  actor_flat = pd.json_normalize(df['actor']).add_prefix('actor.').drop('actor.id', axis=1)
  df = pd.concat([df.drop('actor', axis=1), actor_flat], axis=1)
  repo_flat = pd.json_normalize(df['repo']).add_prefix('repo.').drop('repo.id', axis=1)
  df = pd.concat([df.drop('repo', axis=1), repo_flat], axis=1)
  org_flat = pd.json_normalize(df['org']).add_prefix('org.').drop('org.id', axis=1)
  df = pd.concat([df.drop('org', axis=1), org_flat], axis=1)
  issues_df = df[df['type']=='IssuesEvent']
  issues_df['payload'] = issues_df['payload'].apply(lambda row: format_payload(row))
  payload_flat = pd.json_normalize(issues_df['payload']).add_prefix('payload.')
  issues_df = pd.concat([issues_df.drop('payload', axis=1), payload_flat], axis=1)
  issues_df.to_csv('issues.csv', index=False)

In [7]:
def upload_to_blob_storage( connection_string, container_name, file_path, blob_name):
    blob_service_client = BlobServiceClient.from_connection_string(connection_string)
    blob_client = blob_service_client.get_blob_client(container= container_name, blob= blob_name)

    with open(file_path, "rb") as data:
        blob_client.upload_blob(data)

In [9]:
date = "2023-06-10"
time = "15"
connection_string = "DefaultEndpointsProtocol=https;AccountName=pod4projectstorage;AccountKey=2hClDrVPLGX4QBDBk8OylAkHqczIQfDja66Yl488rmj/0+vb+CAzOxL5qMe5XyM9ZupgwveVRm3N+AStriO5vg==;EndpointSuffix=core.windows.net"
container_name = "data"
file_path = "issues.csv"
blob_name = "issues.csv"

data = get_data(date,time)
normalize_data()
upload_to_blob_storage( connection_string, container_name, file_path, blob_name)

https://data.gharchive.org/2023-06-10-15.json.gz
File downloaded successfully.


<ipython-input-6-e98ab4d5d611>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  issues_df['payload'] = issues_df['payload'].apply(lambda row: format_payload(row))
